In [1]:
import numpy as np
N=50

mat = np.zeros((N,N))
for i in range(N-1):
    mat[i, i+1] = 1
    mat[i+1, i] = 1

print(mat)
print(np.linalg.eig(mat)[0])

def normalize(sigma):
    dim = sigma.shape[0]
    # Impose PD and mean of eigenvalues equal 1

    # Forces all diagonal elements to be the same
    # That's the easiest way to ensure all ev > 0...
    sigma -= np.diag(np.diag(sigma))
    sigma += np.diag([dim for _ in range(dim)])

    sigma *= dim / np.sum(np.linalg.eig(sigma)[0])
    assert np.all(np.linalg.eig(sigma)[0] > 0.)
    assert np.abs(np.sum(np.linalg.eig(sigma)[0]) - dim) < 0.05

    return sigma

mat2 = normalize(mat)
print(mat2)
print(np.linalg.eig(mat2)[0])

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[-1.99620666 -1.98484102 -1.9659462  -1.93959387 -1.905884   -1.86494446
 -1.81693054 -1.76202439 -1.70043427 -1.63239382 -1.55816115 -1.47801783
 -1.39226789 -1.3012366  -1.20526927 -1.10472995 -1.         -0.89147671
 -0.77957175 -0.6647096  -0.54732598 -0.42786617 -0.30678331 -0.18453672
 -0.06159012  0.06159012  0.18453672  0.30678331  0.42786617  0.54732598
  0.6647096   0.77957175  0.89147671  1.99620666  1.98484102  1.9659462
  1.93959387  1.905884    1.86494446  1.81693054  1.76202439  1.70043427
  1.63239382  1.55816115  1.47801783  1.39226789  1.3012366   1.20526927
  1.10472995  1.        ]
[[1.   0.02 0.   ... 0.   0.   0.  ]
 [0.02 1.   0.02 ... 0.   0.   0.  ]
 [0.   0.02 1.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 1.   0.02 0.  ]
 [0.   0.   0.   ... 0.02 1.   0.02]
 [0.   0.   0.   ... 0.   0.02 1.  ]]
[0.9600